In [3]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,auc,roc_curve
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc,f1_score,confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE,ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
RandomUnderSampler
from sklearn  import preprocessing
import  xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from tensorflow import keras as keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# 加载数据集与预处理

In [28]:
#训练集
train=pd.read_csv("./train7.9_screening.csv")
#测试集
test=pd.read_csv("./test7.9_screening.csv")
data = pd.concat([train,test],axis=0,ignore_index=True)
#训练集标签
train_label=pd.read_csv("./y_train.csv")
train_label=train_label.iloc[:,1]

In [29]:
train_data=train.iloc[:,1:].values
train_label=train_label.values.ravel()
print(train_data.shape)
print(train_label.shape)
test_data=test.iloc[:,1:].values
print(test_data.shape)

(784000, 70)
(784000,)
(340043, 70)


In [7]:
model_smote=SMOTE(k_neighbors=20)
X,y=model_smote.fit_resample(train_data,train_label)
X.shape,y.shape

((1400000, 70), (1400000,))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# lgb_1

In [9]:
parameters={
            'objective': 'binary',
            'metric': 'binary_error',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': 1.20240655708393e-08, 
            'lambda_l2': 0.1148043434213944, 
            'num_leaves': 512, 
            'learning_rate': 0.020071584253295194, 
            'n_estimators': 3288, 
            'feature_fraction': 0.9512028862422643, 
            'bagging_fraction': 0.934931341887986, 
            'bagging_freq': 1, 
            'min_child_samples': 29,
            'max_depth':53
            }

In [10]:
gbm = lgb.LGBMClassifier(**parameters)

In [11]:
gbm.fit(X, y,eval_set=[(X,y),(X_test,y_test)],early_stopping_rounds=100)

[LightGBM] [Warning] feature_fraction is set=0.9512028862422643, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9512028862422643
[LightGBM] [Warning] lambda_l1 is set=1.20240655708393e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.20240655708393e-08
[LightGBM] [Warning] bagging_fraction is set=0.934931341887986, subsample=1.0 will be ignored. Current value: bagging_fraction=0.934931341887986
[LightGBM] [Warning] lambda_l2 is set=0.1148043434213944, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1148043434213944
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	training's binary_error: 0.0250093	valid_1's binary_error: 0.0247714
Training until validation scores don't improve for 100 rounds
[2]	training's binary_error: 0.0241207	valid_1's binary_error: 0.0238714
[3]	training's binary_error: 0.0237693	valid_1's binary_error: 0.0236286
[4]	training's binary_error: 0.0233421	va

LGBMClassifier(bagging_fraction=0.934931341887986, bagging_freq=1,
               feature_fraction=0.9512028862422643,
               lambda_l1=1.20240655708393e-08, lambda_l2=0.1148043434213944,
               learning_rate=0.020071584253295194, max_depth=53,
               metric='binary_error', min_child_samples=29, n_estimators=3288,
               num_leaves=512, objective='binary', verbosity=-1)

In [12]:
train_pri1 = gbm.predict_proba(train_data)[:,-1]
train_pri1 = pd.DataFrame(train_pri1,columns=['flag'])
train_pri1.to_csv('./train_lgb_1.csv',index = False)

In [75]:
ind_pri1 = gbm.predict_proba(test_data)[:,-1]
ind_pri1 = pd.DataFrame(ind_pri1,columns=['flag'])
ind_pri1.to_csv('./prob_lgb_1.csv',index = False)

# lgb_2(+focal loss)

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import lightgbm
import os
from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.misc import derivative
import warnings
warnings.filterwarnings('ignore')

In [14]:
# focal loss 损失函数
def focal_loss_lgb_sk(y_true, y_pred, alpha, gamma):
   
    a,g = alpha, gamma
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess

# focal loss 对应的评估函数metric
def focal_loss_lgb_eval_error_sk(y_true, y_pred, alpha, gamma):
    
    a,g = alpha, gamma
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

def sigmoid(x):
    return 1/(1+np.exp(-x))

In [15]:
focal_loss = lambda x,y: focal_loss_lgb_sk(x, y, alpha = 0.4, gamma = 2)

In [16]:
focal_eval = lambda y_ture,y_pred: focal_loss_lgb_eval_error_sk(y_true, y_pred, alpha=0.4, gamma = 2)

In [17]:
parameters={
            'objective': focal_loss,
            'metric': None,
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': 1.20240655708393e-08, 
            'lambda_l2': 0.1148043434213944, 
            'num_leaves': 512, 
            'learning_rate': 0.020071584253295194, 
            'n_estimators': 7000, 
            'feature_fraction': 0.9512028862422643, 
            'bagging_fraction': 0.934931341887986, 
            'bagging_freq': 1, 
            'min_child_samples': 29,
            'max_depth':53
            }

In [18]:
gbm = lgb.LGBMClassifier(**parameters)

In [19]:
gbm.fit(X, y,eval_set=[(X,y),(X_test,y_test)],
        eval_metric=lambda y_true, y_pred: [focal_loss_lgb_eval_error_sk(y_true, y_pred, alpha=0.4, gamma = 2)],
        early_stopping_rounds=100)

[LightGBM] [Warning] feature_fraction is set=0.9512028862422643, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9512028862422643
[LightGBM] [Warning] lambda_l1 is set=1.20240655708393e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.20240655708393e-08
[LightGBM] [Warning] bagging_fraction is set=0.934931341887986, subsample=1.0 will be ignored. Current value: bagging_fraction=0.934931341887986
[LightGBM] [Warning] lambda_l2 is set=0.1148043434213944, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1148043434213944
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	training's binary_logloss: 2.5606	training's focal_loss: 0.0845972	valid_1's binary_logloss: 2.56004	valid_1's focal_loss: 0.0845981
Training until validation scores don't improve for 100 rounds
[2]	training's binary_logloss: 2.19226	training's focal_loss: 0.0826107	valid_1's binary_logloss: 2.19147	valid_1's focal_l

LGBMClassifier(bagging_fraction=0.934931341887986, bagging_freq=1,
               feature_fraction=0.9512028862422643,
               lambda_l1=1.20240655708393e-08, lambda_l2=0.1148043434213944,
               learning_rate=0.020071584253295194, max_depth=53, metric=None,
               min_child_samples=29, n_estimators=7000, num_leaves=512,
               objective=<function <lambda> at 0x000002623080C310>,
               verbosity=-1)

In [22]:
train_pri=gbm.predict(train_data)
train_pri2 = sigmoid(train_pri)
train_pri2 = pd.DataFrame(train_pri2,columns=['flag'])
train_pri2.to_csv('./train_lgb_local.csv',index = False)

In [64]:
ind_pri=gbm.predict(test_data)
ind_pri2 = sigmoid(ind_pri)
ind_pri2 = pd.DataFrame(ind_pri2,columns=['flag'])
ind_pri2.to_csv('./prob_lgb+focal_loss.csv',index = False)

# lgb_3

In [23]:
parameters={ 'objective': 'binary',
             'metric': 'binary_error',
             'verbosity': -1,
             'boosting_type': 'gbdt',
             'lambda_l1': 8.14104577982587e-05, 
             'lambda_l2': 0.25985991684358545, 
             'num_leaves': 425, 
             'learning_rate': 0.029861155556805748, 
             'n_estimators': 3886,
             'feature_fraction': 0.6856367429495369,
             'bagging_fraction': 0.7331422541166238, 
             'bagging_freq': 2, 
             'min_child_samples': 45}

In [24]:
gbm = lgb.LGBMClassifier(**parameters)

In [25]:
gbm.fit(X, y,eval_set=[(X,y),(X_test,y_test)],early_stopping_rounds=100)

[LightGBM] [Warning] feature_fraction is set=0.6856367429495369, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6856367429495369
[LightGBM] [Warning] lambda_l1 is set=8.14104577982587e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.14104577982587e-05
[LightGBM] [Warning] bagging_fraction is set=0.7331422541166238, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7331422541166238
[LightGBM] [Warning] lambda_l2 is set=0.25985991684358545, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.25985991684358545
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[1]	training's binary_error: 0.0463893	valid_1's binary_error: 0.0467464
Training until validation scores don't improve for 100 rounds
[2]	training's binary_error: 0.0259721	valid_1's binary_error: 0.0263464
[3]	training's binary_error: 0.0240521	valid_1's binary_error: 0.0241643
[4]	training's binary_error: 0.023322

LGBMClassifier(bagging_fraction=0.7331422541166238, bagging_freq=2,
               feature_fraction=0.6856367429495369,
               lambda_l1=8.14104577982587e-05, lambda_l2=0.25985991684358545,
               learning_rate=0.029861155556805748, metric='binary_error',
               min_child_samples=45, n_estimators=3886, num_leaves=425,
               objective='binary', verbosity=-1)

In [26]:
train_pri1 = gbm.predict_proba(train_data)[:,-1]
train_pri1 = pd.DataFrame(train_pri1,columns=['flag'])
train_pri1.to_csv('./train_lgb_3.csv',index = False)

In [114]:
ind_pri3 = gbm.predict_proba(test_data)[:,-1]

In [115]:
ind_pri3 = pd.DataFrame(ind_pri3,columns=['flag'])
ind_pri3.to_csv('prob_lgb_2.csv',index = False)